In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.model_selection import btrain_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [9]:
score = 1
np.array([score])
pd.DataFrame(np.array([score]))

,0
0,1


In [65]:
def stacking(clf, X_train, y_train, X_test, n_folds=10):
    train_num = X_train.shape[0]
    test_num = X_test.shape[0]
    
    second_level_train_set = np.zeros((train_num,))
    second_level_test_set = np.zeros((test_num,))
    
    test_nfolds_sets = np.zeros((test_num, n_folds))
    
    kf = KFold(n_splits=n_folds, random_state=2018)
    
    for i, (train_index, test_index) in enumerate(kf.split(X_train)):
        
        X_tra = X_train[train_index]
        y_tra = y_train[train_index]
        
        X_tst = X_train[test_index]
        y_tst = y_train[test_index]
        
        clf.fit(X_tra, y_tra)
        
        second_level_train_set[test_index] = clf.predict(X_tst)
        
        test_nfolds_sets[:, i] = clf.predict(X_test)
        
    second_level_test_set[:] = test_nfolds_sets.mean(axis=1)
    
    return second_level_train_set.reshape(-1, 1), second_level_test_set.reshape(-1, 1)

In [146]:
rf = RandomForestClassifier()
ada = AdaBoostClassifier()
gbdt = GradientBoostingClassifier()

In [147]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=2018)
print(X_train.shape)
print(X_test.shape)

(120, 4)
(30, 4)


In [148]:
train_set1, test_set1 = stacking(rf, X_train, y_train, X_test)
print(train_set1.shape)
print(test_set1.shape)

(120, 1)
(30, 1)


In [149]:
train_set2, test_set2 = stacking(ada, X_train, y_train, X_test)
print(train_set2.shape)
print(test_set2.shape)

(120, 1)
(30, 1)


In [150]:
train_set3, test_set3 = stacking(gbdt, X_train, y_train, X_test)
print(train_set3.shape)
print(test_set3.shape)

(120, 1)
(30, 1)


In [151]:
train_sets = []
test_sets = []
train_sets.append(train_set1)
train_sets.append(train_set2)
train_sets.append(train_set3)
test_sets.append(test_set1)
test_sets.append(test_set2)
test_sets.append(test_set3)

In [152]:
meta_train = np.concatenate([result_set for result_set in train_sets], axis=1)
meta_test = np.concatenate([result_set for result_set in test_sets], axis=1)

In [153]:
print(meta_train.shape)
print(meta_test.shape)

(120, 3)
(30, 3)


In [154]:
dt = DecisionTreeClassifier()
dt.fit(meta_train, y_train)
y_pred = dt.predict(meta_test)

In [155]:
accuracy_score(y_test, y_pred)

0.9666666666666667